In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Hackathon Master Data Sciences & Business Analytics

Don't forget to enable GPU on the running instance to run it faster (Execution > Modify execution type)

# Install some libraries

In [ ]:
%pip install langchain langchain-community  gradio sentence-transformers tiktoken langchain-groq faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1

# Import some libraries

In [ ]:

import shutil
import pickle
import tempfile
from operator import itemgetter
import time
from bs4 import BeautifulSoup
import pandas as pd
import gradio as gr

import os
import yaml
from tqdm import tqdm
from langchain.prompts.prompt import PromptTemplate
from typing import Tuple, List
from langchain.schema.runnable import RunnableMap
from langchain.schema import format_document
from datetime import datetime
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

from langchain.document_loaders import DataFrameLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS
from langchain.schema import Document, HumanMessage, BaseMessage
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackManager
from langchain.globals import set_llm_cache

from langchain_groq import ChatGroq


# Define some functions

In [ ]:
def _combine_documents(
    docs, document_prompt, document_separator="\n\n"
):
    doc_strings = [
        f"Document {i}: \n'''\n{format_document(doc, document_prompt)}\n'''"
        for i, doc in enumerate(docs, 1)
    ]
    return document_separator.join(doc_strings)


def _format_chat_history(chat_history: List[Tuple]) -> str:
    turn = 1
    buffer = []
    for dialogue in chat_history:
        buffer.append(("Human: " if turn else "Assistant: ") + dialogue.content)
        turn ^= 1
    return "\n".join(buffer) + "\n"


def make_pairs(lst):
    """from a list of even lenght, make tupple pairs"""
    return [(lst[i], lst[i + 1]) for i in range(0, len(lst), 2)]


def make_html_source(i, doc):
    return f"""
<div class="card">
  <div class="card-content">
      <h3>Doc {i}</h2>
      <p>{BeautifulSoup(doc.page_content, 'html.parser')}</p>
  </div>
  <div class="card-footer">
    <span>page: {doc.metadata['page_number']}</span>
  </div>
</div>
"""

async def chat(
    query: str,
    history: list = [],
    length: str = "Medium",
    language: str = "the same language of the question",
    audience: str = "public",
    tone: str = "formal"
):
    """taking a query and a message history, use a pipeline (reformulation, retriever, answering) to yield a tuple of:
    (messages in gradio format, messages in langchain format, source documents)"""
    source_string = ""
    gradio_format = make_pairs([a.content for a in history]) + [(query, "")]

    # reset memory
    memory.clear()
    for message in history:
        memory.chat_memory.add_message(message)

    inputs = {"question": query, "length": length, "language": language, "audience": audience, "tone": tone}
    result = final_chain.astream_log(inputs)

    reformulated_question_path_id = "/logs/ChatGroq/streamed_output_str/-"
    retriever_path_id = "/logs/Retriever/final_output"
    final_answer_path_id = "/logs/ChatGroq:2/streamed_output_str/-"

    async for op in result:
        op = op.ops[0]
        if op["path"] == reformulated_question_path_id:  # reforulated question
            new_token = op["value"]  # str

        elif op["path"] == retriever_path_id:  # documents
            sources = op["value"]["documents"]  # List[Document]
            source_string = "\n\n".join(
                [make_html_source(i, doc) for i, doc in enumerate(sources, 1)]
            )

        elif op["path"] == final_answer_path_id:  # final answer
            new_token = op["value"]  # str
            answer_yet = gradio_format[-1][1]
            gradio_format[-1] = (query, answer_yet + new_token)

        yield "", gradio_format, history, source_string

    memory.save_context(inputs, {"answer": gradio_format[-1][1]})
    yield "", gradio_format, memory.load_memory_variables({})["history"], source_string

# Define global params

In [ ]:
demo_name = "Hackathon climate Q-A"

# Put your token from https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "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"

# Treat data

## Load and prepare parsed data

In [ ]:
documents = []
path_data = "/content/drive/MyDrive/Group 6/data/" # maybe need to be adapted

In [ ]:
for file in tqdm(os.listdir(path_data)):
  if file.endswith('.parquet'):
    df = pd.read_parquet(os.path.join(path_data, file))
    df = df[df["sub_type"] == "Text"]
    df = df[["content", "file_name", "page_number"]]
    loader = DataFrameLoader(df, page_content_column="content")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256, chunk_overlap=50, length_function=lambda x: len(x.split())
    )

  docs = loader.load_and_split(text_splitter)
  documents.extend(docs)

100%|██████████| 15/15 [00:40<00:00,  2.69s/it]


## Get embeddings, vectorstore and retriever

In [ ]:
model_name = "BAAI/bge-small-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
print("Loading embeddings model: ", model_name)
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages: "
)

Loading embeddings model:  BAAI/bge-small-en


In [ ]:
# Around 3mins with GPU enabled
vectorstore = FAISS.from_documents(
    documents, embedding=embeddings,
)

retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("What is climate change ?")

[Document(page_content='Climate Change. https://doi.org/10.1038/\r\nnclimate3382', metadata={'file_name': '202206-IPBES-GLOBAL-REPORT-FULL-DIGITAL-MARCH-2022.pdf', 'page_number': 786}),
 Document(page_content="Climate change A change in the state of the climate that can be \r\nidentified (e.g., by using statistical tests) by changes in the mean and/\r\nor the variability of its properties and that persists for an extended \r\nperiod, typically decades or longer. Climate change may be due to \r\nnatural internal processes or external forcings such as modulations \r\nof the solar cycles, volcanic eruptions and persistent anthropogenic\r\nchanges in the composition of the atmosphere or in land use. Note \r\nthat the United Nations Framework Convention on Climate Change \r\n(UNFCCC), in its Article 1, defines climate change as: 'a change of \r\nclimate which is attributed directly or indirectly to human activity \r\nthat alters the composition of the global atmosphere and which is \r\nin a

## Get LLM from Groq

In [ ]:
chat_model = ChatGroq(temperature=0, model_name="llama3-8b-8192") # llama3-70b-8192 / mixtral-8x7b-32768

## Create a prompt

In [ ]:
reformulation_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(reformulation_template)

answering_template = """
  You are ClimateQ&A, an AI Assistant created by Ekimetrics. You are given a question and extracted passages of the IPCC and/or IPBES reports. Provide a clear and structured answer based on the passages provided, the context and the guidelines.

  Guidelines:
  - If the passages have useful facts or numbers, use them in your answer.
  - When you use information from a passage, mention where it came from by using [Doc i] at the end of the sentence. i stands for the number of the document.
  - Do not use the sentence 'Doc i says ...' to say where information came from.
  - If the same thing is said in more than one document, you can mention all of them like this: [Doc i, Doc j, Doc k]
  - Do not just summarize each passage one by one. Group your summaries to highlight the key parts in the explanation.
  - If it makes sense, use bullet points and lists to make your answers easier to understand.
  - You do not need to use every passage. Only use the ones that help answer the question.
  - If the documents do not have the information needed to answer the question, just say you do not have enough information.
  - Consider by default that the question is about the past century unless it is specified otherwise.
  - If the passage is the caption of a picture, you can still use it as part of your answer as any other document.

  Your answer should be {length}.
  Answer in {language}.
  You should have an {tone} tone.
  You should speak as if you were speaking to {audience}
  -----------------------
  Passages:
  {context}

  -----------------------
  Question: {question}
  Answer with the passages citations:
  """

ANSWER_PROMPT = ChatPromptTemplate.from_template(answering_template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}.\page: {page_number}")

## Chain some functions

In [ ]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | chat_model
    | StrOutputParser(),
    "length": lambda x: x["length"],
    "language": lambda x: x["language"],
    "audience": lambda x: x["audience"],
    "tone": lambda x: x["tone"],
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
    "length": lambda x: x["length"],
    "language": lambda x: x["language"],
    "audience": lambda x: x["audience"],
    "tone": lambda x: x["tone"],
}

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"], DEFAULT_DOCUMENT_PROMPT),
    "question": itemgetter("question"),
    "length": itemgetter("length"),
    "language": itemgetter("language"),
    "audience": itemgetter("audience"),
    "tone": itemgetter("tone")
}

# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | chat_model,
    "docs": itemgetter("docs"),
}

# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

# Setup your Gradio

## Get CSS styling

In [ ]:
with open("./drive/MyDrive/Group 6/Assets/style.css", "r") as f:# maybe path need to be adapted
    css = f.read()

## Create your app

In [ ]:
with gr.Blocks(title=f"{demo_name}", css=css) as demo:

    gr.Markdown(f"<h1><center>{demo_name}</center></h1>")

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                elem_id="chatbot", label=f"{demo_name} chatbot", show_label=False
            )
            state = gr.State([])

            # Add the drop-down box here
            length = gr.Dropdown(
                choices=["Very short", "Short", "Medium", "Long", "Very long"],
                label="Select response length"
            )

            language = gr.Dropdown(
                choices=["English", "French", "Italian", "Spanish"],
                label="Select language",
                value="the same language of the question"
            )

            audience = gr.Dropdown(
                choices=["Children", "Public", "Experts"],
                label="Select audience"
            )

            tone = gr.Dropdown(
                choices=["Informal", "Formal", "Dramatic", "Fun"],
                label="Select tone"
            )

            with gr.Row():
                ask = gr.Textbox(
                    show_label=False,
                    placeholder="Input your question then press enter",
                )

        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### Sources")
            sources_textbox = gr.Markdown(show_label=False)

        ask.submit(
            fn=chat,
            inputs=[
                ask,
                state,
                length,
                language,
                audience,
                tone
            ],
            outputs=[ask, chatbot, state, sources_textbox],
        )

demo.launch(
    share=True,
    # auth=("", ""),
    debug=True
)

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:181: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: the same language of the question or set allow_custom_value=True.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://85012fb7166d393245.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<ipython-input-62-948639ad7765>:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  <p>{BeautifulSoup(doc.page_content, 'html.parser')}</p>
<ipython-input-62-948639ad7765>:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  <p>{BeautifulSoup(doc.page_content, 'html.parser')}</p>
<ipython-input-62-948639ad7765>:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  <p>{BeautifulSoup(doc.page_content, 'html.parser')}</p>
<ipython-input-62-948639ad7765>:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  <p>{BeautifulSoup(doc.page_content, 'html.parser')}<